In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00


In [ ]:
!pip install conllu

In [ ]:
!pip install sentencepiece

In [ ]:
def decode_tokens(example):
    idx = 1
    enc =[(x, tokenizer.encode(x, add_special_tokens=False)) for x in example.split()]

    desired_output = []

    for x, token in enc:
        tokenoutput = []
        for ids in token:
            tokenoutput.append(idx)
            idx +=1
        desired_output.append(tokenoutput)

    return desired_output

In [ ]:
from transformers import MT5Model, T5Tokenizer
import pandas as pd
import torch
from tqdm.auto import tqdm

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("google/mt5-base")
model = MT5Model.from_pretrained("google/mt5-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Hindi-HDTB/master/hi_hdtb-ud-train.conllu

--2024-03-13 05:02:41--  https://raw.githubusercontent.com/UniversalDependencies/UD_Hindi-HDTB/master/hi_hdtb-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41232818 (39M) [text/plain]
Saving to: ‘hi_hdtb-ud-train.conllu’

hi_hdtb-ud-train.co 100%[===================>]  39.32M  --.-KB/s    in 0.1s    

2024-03-13 05:02:43 (270 MB/s) - ‘hi_hdtb-ud-train.conllu’ saved [41232818/41232818]



In [15]:
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Hindi-HDTB/master/hi_hdtb-ud-dev.conllu

--2024-03-13 05:08:06--  https://raw.githubusercontent.com/UniversalDependencies/UD_Hindi-HDTB/master/hi_hdtb-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5161765 (4.9M) [text/plain]
Saving to: ‘hi_hdtb-ud-dev.conllu’

hi_hdtb-ud-dev.conl 100%[===================>]   4.92M  --.-KB/s    in 0.1s    

2024-03-13 05:08:07 (50.3 MB/s) - ‘hi_hdtb-ud-dev.conllu’ saved [5161765/5161765]



In [ ]:

!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Bhojpuri-BHTB/master/bho_bhtb-ud-test.conllu

--2024-03-13 05:02:48--  https://raw.githubusercontent.com/UniversalDependencies/UD_Bhojpuri-BHTB/master/bho_bhtb-ud-test.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 859481 (839K) [text/plain]
Saving to: ‘bho_bhtb-ud-test.conllu’

bho_bhtb-ud-test.co 100%[===================>] 839.34K  --.-KB/s    in 0.03s   

2024-03-13 05:02:48 (26.2 MB/s) - ‘bho_bhtb-ud-test.conllu’ saved [859481/859481]



In [ ]:
from datasets import Dataset
import conllu

In [ ]:
def get_dataset(filename):
    with open(filename) as file:
        text = file.read()

    sents = {"text":[], "tag":[]}
    for sent in conllu.parse(text):
        tokens, pos = list(zip(*[(token["form"], token["upos"]) for token in sent if token["upos"] != "_"]))
        sents["text"].append(" ".join(tokens))
        sents["tag"].append(" ".join(pos))

    dataset = Dataset.from_dict(sents)

    return dataset

In [16]:
train_dataset = get_dataset("hi_hdtb-ud-train.conllu")
eval_dataset = get_dataset("hi_hdtb-ud-dev.conllu")
test_dataset = get_dataset("bho_bhtb-ud-test.conllu")

In [17]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4)
eval_loader = torch.utils.data.DataLoader(eval_dataset, batch_size=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4)

In [18]:
device = "cuda:0"

model = model.to(device)

In [19]:
def parse_batch(batch, sentences, length=768):
    embedding_list = []
    tags = []
    for embedding, sentence, tag in zip(batch, sentences["text"], sentences["tag"]):
        tokens = decode_tokens(sentence)
        embeddings = torch.zeros(len(tokens), length)
        for num, token in enumerate(tokens):
            embeddings[num,:] = embedding[token[0]:token[-1]+1].mean(axis=0)
        embedding_list.append(embeddings)
        tags.extend([word for word in tag.split()])
    return torch.cat(embedding_list), tags

In [20]:
class LogReg(torch.nn.Module):
    def __init__(self, input_dim: int, num_classes: int):
        super(LogReg, self).__init__()
        self.input_dim = input_dim
        self.num_classes = num_classes
        self.fc = torch.nn.Linear(self.input_dim, self.num_classes)
        self.activation = torch.nn.Sigmoid()

    def forward(self, x: torch.Tensor):
        x = self.fc(x)
        #x = self.activation(x)
        return x

In [21]:
class VectorDataset(torch.utils.data.Dataset):
    def __init__(self, embeddings, labels):
        self.input_ids = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx: int):
        input_ids = self.input_ids[idx]
        labels = self.labels[idx]
        sample = (input_ids, labels)
        return sample

In [22]:
def get_vector_dataset(train_loader):
    labels = []
    embeddings = []
    for sentence in tqdm(train_loader):
        enc = tokenizer(sentence["text"], padding=True, truncation=True,
                        return_tensors="pt")
        with torch.no_grad():
            enc = enc.to(device)
            output = model(**enc, decoder_input_ids=enc.input_ids, return_dict=True)
            embs, tags = parse_batch(output["last_hidden_state"].detach().cpu(), sentence)
            embeddings.append(embs)
            labels.extend(tags)

    return torch.cat(embeddings), labels

In [23]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
train_embs, train_labels = get_vector_dataset(train_loader)
eval_embs, eval_labels = get_vector_dataset(eval_loader)
test_embs, test_labels = get_vector_dataset(test_loader)

encoder = LabelEncoder()

labels = train_labels + test_labels
encoder.fit(labels)

train_labels = encoder.transform(train_labels)
train_data = VectorDataset(train_embs, train_labels)
pos_train = torch.utils.data.DataLoader(train_data, batch_size=8)


eval_labels = encoder.transform(eval_labels)
eval_data = VectorDataset(eval_embs, eval_labels)
pos_eval = torch.utils.data.DataLoader(eval_data, batch_size=8)


test_labels = encoder.transform(test_labels)
test_data = VectorDataset(test_embs, test_labels)
pos_test = torch.utils.data.DataLoader(test_data, batch_size=8)

  0%|          | 0/3327 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
embedding, label = next(iter(pos_train))
input_dim = embedding.shape[1]

In [ ]:
embedding.shape

In [ ]:
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
import numpy as np

In [ ]:
classifier = LogReg(input_dim=input_dim, num_classes=len(encoder.classes_))
optimizer = AdamW(classifier.parameters(), lr=0.001)
criterion = CrossEntropyLoss().to(device)

In [ ]:
epochs=50
epoch_train_losses = []
classifier.train()
classifier.to(device)
for epoch in tqdm(range(epochs)):
    for i, batch in enumerate(pos_train):
            # x is already on device since it was passed through the model
                y = batch[1].to(device, non_blocking=True)

                x = batch[0].to(device)

                classifier.zero_grad(set_to_none=True)
                prediction = classifier(x)
                loss = criterion(prediction, y)
                epoch_train_losses.append(loss.item())
                loss.backward()

                if (i + 1) % 2 == 0 or (i + 1) == len(pos_train):
                    optimizer.step()

    if (epoch + 1) % 10 == 0:
        epoch_loss = np.mean(epoch_train_losses)
        print(f"Epoch {epoch}: loss {epoch_loss}")

In [ ]:
epoch_losses = []
epoch_predictions = []
epoch_true_labels = []

classifier.eval()
with torch.no_grad():
        for x, y in pos_eval:
                # x is already on device since it was passed through the model
                y = y.to(device, non_blocking=True)

                x = x.to(device)
               # x = torch.squeeze(x[layer], 0).float()
                #x = torch.unsqueeze(x, 0) if len(x.size()) == 1 else x

                prediction = classifier(x)
                loss = criterion(prediction, y)
                epoch_losses.append(loss.item())

                epoch_predictions += prediction.cpu().data.max(1).indices
                epoch_true_labels += y.cpu()

epoch_loss = np.mean(epoch_losses)


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
accuracy_score(epoch_true_labels, epoch_predictions)

In [ ]:
f1_score(epoch_true_labels, epoch_predictions, average="weighted")

In [ ]:
epoch_losses = []
epoch_predictions = []
epoch_true_labels = []

classifier.eval()
with torch.no_grad():
        for x, y in pos_test:
                # x is already on device since it was passed through the model
                y = y.to(device, non_blocking=True)

                x = x.to(device)
               # x = torch.squeeze(x[layer], 0).float()
                #x = torch.unsqueeze(x, 0) if len(x.size()) == 1 else x

                prediction = classifier(x)
                loss = criterion(prediction, y)
                epoch_losses.append(loss.item())

                epoch_predictions += prediction.cpu().data.max(1).indices
                epoch_true_labels += y.cpu()

epoch_loss = np.mean(epoch_losses)


In [ ]:
accuracy_score(epoch_true_labels, epoch_predictions)

In [ ]:
f1_score(epoch_true_labels, epoch_predictions, average="weighted")